## Compute binding frequency matrix

Use this routine to compute a tensor of binding frequencies from trajectories with different (fixed) numbers of reagent copies. Each entry (ijklmn) of the tensor gives you the frequency (relative number of trajectory frames) with which:

    -nucleotide i is in binding state m (1 if bound, 0 if not)
    -nucleotide j is in binding state n (1 if bound, 0 if not)
    -k reagent copies are in region A (binding region)
    -l reagent copies are in region B (buffer region)

In [1]:
# Import required modules

import numpy as np
import pandas as pd
import mdtraj as md
import subprocess as sp
from scipy.spatial import distance

import os
import itertools
from random import choices
import time
import multiprocess as mp

In [2]:
# Function to compute binding tensor from a single trajectory

def compute_bindings(traj, # mdtraj trajectory
                              top, # mdtraj topology
                              nprobes, # number of reagent copies in the trajectory
                              maxnprobes, # maximum number of reagent copies in the analysis
                              max_binding_distance, # binding distance threshold
                              binding_region_radius): # radius of the binding region
    
    # Compute distances between all pairs of atoms involved in binding (O2 in nucleotides, C7 in 1M7)
    
    o2p = top.select('name== "O2\'"')
    c7 = top.select('name == "C7"')
    pairs_tocheck = np.array([(x,y) for x in o2p for y in c7])
    dist = md.compute_distances(traj, pairs_tocheck)
    
    # Reshape distance matrix as (number of frames x number of RNA sites x number of reagent copies)
    
    nframes = dist.shape[0]
    nsites = int(dist.shape[1]/nprobes)
    dist = dist.reshape((nframes,nsites,nprobes))
    
    # Identify 1M7 and RNA atoms 
    
    atoms_per_probe = np.array_split(top.select("resname =~ '1M7'"), nprobes)
    rna_atoms = top.select("(resname =~ 'A') or (resname =~ 'U') or (resname =~ 'C') or (resname =~ 'G')")
    
    # Initialize binding tensor entries to zero

    binding_tensor = np.zeros((nsites,nsites,maxnprobes+1,maxnprobes+1,2,2))
    
    # Compute the RNA center of mass in each trajectory frame
    
    cm_rna = md.compute_center_of_mass(traj.atom_slice(rna_atoms))
    
    # Compute number of reagent copies in binding (na) and buffer (nb) regions in each trajectory frame
    
    na = np.zeros(nframes)
    nb = np.zeros(nframes)
    probes_starting_atoms = [atoms[0] for atoms in atoms_per_probe]
    probes_ending_atoms = [atoms[-1] for atoms in atoms_per_probe]
    for probe in range(nprobes):
        # Compute distance between the centers of mass of reagent and RNA for each trajectory frame
        probe_atoms = traj.atom_slice(list(range(probes_starting_atoms[probe], probes_ending_atoms[probe]+1)))
        cm = md.compute_center_of_mass(probe_atoms)-cm_rna
        r = np.sqrt(np.sum((cm**2), axis = 1))
        # Increment na and nb of each trajectory frame if the reagent is the corresponding region
        na += (r < binding_region_radius)
        nb += (r >= binding_region_radius)
    
    # Turn NA and NB values to integer
    
    na = na.astype(int)
    nb = nb.astype(int)
    
    # At each trajectory frame compute the distance between each probe and its closest nucleotide 
    # Find closest nucleotides for each time and probe
    closest_sites = np.argmin(dist, axis=1)
    D = np.zeros(dist.shape) # auxiliary binding matrix (number of frames x number of RNA sites x number of reagent copies)
    for t in range(nframes):
        for (r,i) in enumerate(closest_sites[t]):
            if dist[t,i,r] <= max_binding_distance:
                D[t,i,r] = 1
        # For each pair of nucleotides, compute occurrences of 
        # possible binding states (both bound, both unbound, only one bound)
        for i in range(nsites):
            for j in range(nsites):
                if np.max(D[t,i]) == np.max(D[t,j]):
                    if np.max(D[t,i]) == 0:
                        binding_tensor[i,j,na[t],nb[t],0,0] += 1
                    else:
                        binding_tensor[i,j,na[t],nb[t],1,1] += 1
                elif np.max(D[t,i]) != np.max(D[t,j]):
                    if np.max(D[t,i]) == 0:
                        binding_tensor[i,j,na[t],nb[t],0,1] += 1
                    else:
                        binding_tensor[i,j,na[t],nb[t],1,0] += 1

    # Divide occurrences by totat number of frames to get frequencies
    
    binding_tensor /= nframes 
    return(binding_tensor)

In [3]:
# Routine to parallel-compute the binding tensor from multiple trajectories (different reagent copy numbers)

min_nprobes = 1
max_nprobes = 10 

max_binding_distance = 0.35 # 3.5 Å
binding_region_radius = 2.9 # 290 Å

def worker(nprobes):
    
    # Initialize binding tensor with 0 entries
    binding_tensor = np.zeros((10,10,max_nprobes,max_nprobes,2,2))
    
    # Load topology
    top = md.load_topology('../data/example_data/topologies/top_n'+str(nprobes)+'.gro')
    top = top.subset(top.select("(resname =~ '1M7') or (resname =~ 'A') or (resname =~ 'U') or (resname =~ 'C') or (resname =~ 'G')"))
    
    # Load trajectory
    traj_filename = '../data/example_data/trajectories/traj_n'+str(nprobes)+'_reduced.xtc'
    traj = md.load(traj_filename, top = top)

    # Iteratively load 10000-frames blocks of trajectory and compute binding tensor
    # Exclude the first 30000 trajectory frames from analysis to avoid inital configuration biases
    binding_tensor = compute_bindings(traj, 
                                      top, 
                                      nprobes, 
                                      max_nprobes, 
                                      max_binding_distance, 
                                      binding_region_radius)
    return binding_tensor

In [4]:
## Example run on example data

pool = mp.Pool(10)
result = pool.map(worker,(list(range(1,10))))